In [ ]:
import pdfplumber
import pandas as pd
import os

def extract_tables_from_pdf(pdf_path):
    all_extracted_data = []
    pending_row = None
    previous_headers = None

    def clean_text(text):
        if pd.isnull(text):
            return ''
        return str(text).strip().lower().replace('\n', ' ')

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            tables = page.extract_tables()

            for table in tables:
                df = pd.DataFrame(table).applymap(clean_text)

        
                # Gabungkan baris yang terpotong antar halaman
                if pending_row is not None:
                    df.iloc[0] = df.iloc[0].combine_first(pending_row)
                    pending_row = None
                
                # Simpan baris terakhir jika tidak lengkap
                last_row = df.iloc[-1]
                if last_row.isnull().sum() > 0:
                    pending_row = last_row
                    df = df[:-1]

                all_extracted_data.append(df)

    return all_extracted_data


def process_all_pdfs_in_folder(folder_path, output_excel):
    all_data = []

    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, file)
            extracted_data = extract_tables_from_pdf(pdf_path)
            
            if extracted_data:
                all_data.extend(extracted_data)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_excel(output_excel, index=False, header=True)
        print(f"✅ Semua tabel berhasil diekspor ke {output_excel}")
    else:
        print("⚠ Tidak ada tabel yang cocok ditemukan.")

# Contoh penggunaan
folder_path = r"C:\Users\u532419\Downloads\Data Uang Meragukan BI\File Gabungan Semua"  # Ganti dengan folder tempat PDF berada
output_excel = "output_tabel.xlsx"
process_all_pdfs_in_folder(folder_path, output_excel)

Extracted Table:
['tanggal cara te', 'muan', 'waktu nam', 'a', 'kantor', 'provinsi kota', '']
['temuan', '', 'pendeteksian', '', '', '', '']
['11-02-2025 layana', 'n teller', 'setelah kcu', 'te', 'luk', 'dki jakarta jaku', 't']
['', '', '', '', '', '', '']
['jenis kantor', 'nama', 'dokume no.', 'ke', 'terang', 'provin kota', 'keca']
['kontribu kontribut', 'kontribut', 'n identit', 'an', '', 'si', 'matan']
['tor or', 'or', 'penduku as', '', '', '', '']
['', '', 'ng', '', '', '', '']
['', '', '', '', '', '', '']
['nasabah', 'agus', 'ktp 83278', 'jl.', '', 'jawa solo', 'anyar']
['', '', '23', 'pa', 'hlawa', '', '']
['', '', '', 'n n', 'o 50', '', '']
['pecahan tahun em', 'isi no. s', 'eri 1 no. seri 2', '', 'jumlah', 'jumlah su', 'btotal']
['', '', '', '', 'lembar', 'lembar', '']
['', '', '', '', '', 'terima', '']
['50,000 2022', 'za83', '743 za83742', '', '1', '1 50', ',000']
['jumlah dianalisa', '', 'hasil analisa', '', '', 'subtotal', '']
['', '', '', '', '', '', '']
['1', '', 'palsu',

IndexError: list index out of range

In [ ]:
import pandas as pd

# Fungsi untuk menggeser nilai di bawah header yang tidak sejajar secara otomatis dan menghapus sel salah

def perbaiki_nilai_tidak_sejajar(df):
    kasus_ditemukan = False
    
    # Loop untuk memproses semua pasangan header dan nilai
    for i in range(len(df) - 1):
        if 'jumlah dianalisa' in df.iloc[i].values:
            header_row = i
            value_row = i + 1
            
            # Proses hanya jika ada baris nilai setelah header
            if value_row < len(df):
                kasus_ditemukan = True
                header_positions = {}
                nilai_tersedia = []
                
                # Cari posisi header dan semua nilai yang ada
                for col in range(len(df.columns)):
                    header = df.iloc[header_row, col]
                    value = df.iloc[value_row, col]
                    
                    if pd.notna(header):
                        header_positions[col] = header
                    if pd.notna(value):
                        nilai_tersedia.append(value)
                
                # Geser nilai ke kolom yang sesuai, urut dari kiri ke kanan, lalu hapus yang salah
                nilai_index = 0
                for col in header_positions:
                    if nilai_index < len(nilai_tersedia):
                        df.iloc[value_row, col] = nilai_tersedia[nilai_index]
                        nilai_index += 1
                    else:
                        df.iloc[value_row, col] = None  # Hapus data yang salah
    
    if not kasus_ditemukan:
        print("Tidak ada kasus nilai tidak sejajar ditemukan.")
    
    return df


# Contoh penggunaan:
file_path = "output_tabel.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1', header=None)

# Perbaiki semua kasus nilai yang tidak sejajar
df_diperbaiki = perbaiki_nilai_tidak_sejajar(df)

df_diperbaiki.to_excel("output_tabel.xlsx", index=False)

In [ ]:
import pandas as pd
from openpyxl import load_workbook

# Baca file Excel
file_path = 'output_tabel.xlsx'
sheet1 = pd.read_excel(file_path, sheet_name='Sheet1', header=None)

# Inisialisasi list untuk menyimpan data
data_rows = []
current_row = {}
cols_needed = [
    "tanggal temuan", "cara temuan", "waktu pendeteksian", "nama kantor", "provinsi", "kota",
    "jenis kontributor", "kantor kontributor", "nama kontributor", "dokumen pendukung", 
    "no. identitas", "keterangan", "provinsi", "kota", "kecamatan", "pecahan", "tahun emisi",
    "no. seri 1", "no. seri 2", "jumlah lembar", "jumlah lembar terima", "subtotal",
    "jumlah dianalisa", "hasil analisa", "subtotal"
]

# Variabel tambahan untuk memastikan setiap tabel memiliki batasan yang jelas
found_first_subtotal = False

# Proses parsing data
for idx, row in sheet1.iterrows():
    for col_num, cell_value in enumerate(row):
        if cell_value in cols_needed:
            # Ambil nilai di bawah header jika ada
            if idx+1 < len(sheet1):
                next_val = sheet1.iloc[idx+1, col_num]
                
                # Penanganan khusus untuk "subtotal"
                if cell_value == 'subtotal':
                    if found_first_subtotal:
                        current_row['subtotal 2'] = next_val
                        data_rows.append(current_row)  # Simpan data sebagai satu tabel
                        current_row = {}  # Reset untuk tabel berikutnya
                        found_first_subtotal = False  # Reset flag subtotal pertama
                    else:
                        current_row['subtotal'] = next_val
                        found_first_subtotal = True
                elif cell_value == 'provinsi' and 'provinsi' in current_row:
                    current_row['provinsi_kontributor'] = next_val
                elif cell_value == 'kota' and 'kota' in current_row:
                    current_row['kota_kontributor'] = next_val
                else:
                    current_row[cell_value] = next_val

# Pastikan tabel terakhir tersimpan jika tidak ada subtotal kedua
if current_row:
    data_rows.append(current_row)

# Buat DataFrame
df = pd.DataFrame(data_rows)

# Cek apakah ada tabel tanpa "tanggal temuan", lalu pindahkan provinsi & kota ke provinsi_kontributor & kota_kontributor
if 'tanggal temuan' in df.columns:
    df['tanggal temuan'] = pd.to_datetime(df['tanggal temuan'], errors='coerce')
    mask_no_date = df['tanggal temuan'].isna()
    df.loc[mask_no_date, 'provinsi_kontributor'] = df.loc[mask_no_date, 'provinsi']
    df.loc[mask_no_date, 'kota_kontributor'] = df.loc[mask_no_date, 'kota']
    df.loc[mask_no_date, ['provinsi', 'kota']] = None  # Kosongkan kolom provinsi & kota jika "tanggal temuan" tidak ada

# Format ulang "tanggal temuan"
date_columns = ["tanggal temuan"]
for col in date_columns:
    if col in df.columns:
        df[col] = df[col].dt.strftime('%d-%m-%Y')

df.fillna(method='ffill', inplace=True)

# Simpan ke sheet baru
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    df.to_excel(writer, sheet_name='Sheet2', index=False)

print("Sheet2 berhasil dibuat dengan struktur yang diminta!")

Sheet2 berhasil dibuat dengan struktur yang diminta!


C:\Users\lenovo\AppData\Local\Temp\ipykernel_30660\3367669061.py:56: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['tanggal temuan'] = pd.to_datetime(df['tanggal temuan'], errors='coerce')
C:\Users\lenovo\AppData\Local\Temp\ipykernel_30660\3367669061.py:68: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
